In [15]:
from utility.postgres_manager import *
from utility.document_streaming import *
postgresManager = PostgresManager()
documentStreaming = PostgresStreaming()


In [16]:
import csv

categories_meta = []
categories_keywords = {}
with open('./utility/files/categories_keywords.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    next(readCSV, None)
    for row in readCSV:
        categories_keywords[row[0]] = []
with open('./utility/files/categories_keywords.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    next(readCSV, None)       
    for row in readCSV:
        categories_keywords[row[0]].append(row[2])


In [17]:
for key, value in categories_keywords.items():
    meta = {'id':key, 'text':documentStreaming.tokenize(documentStreaming.process_document(value))}
    categories_meta.append(meta)


In [18]:
from flask_app.utility.documents_gensim import *
import numpy as np
corpus_path = "./gensim_data/updated_subjects/politic-ai-corpus.mm"
dictionary_path = corpus_path + ".dict"
dictionary = gensim.corpora.Dictionary.load(dictionary_path)
corpus = gensim.corpora.MmCorpus("./gensim_data/dossier_corpus.mm")

In [19]:
categories_corpus = []
for key, value in categories_keywords.items():
    categories_meta[len(categories_corpus)] = key
    categories_corpus.append(dictionary.doc2bow(value))


In [20]:
gensim.corpora.MmCorpus.serialize("./gensim_data/categories_corpus.mm", categories_corpus)
categories_corpus = gensim.corpora.MmCorpus("./gensim_data/categories_corpus.mm")

In [21]:
tfidf = TfidfModel(categories_corpus, id2word=dictionary)
corpus_tfidf = tfidf[categories_corpus]
index = gensim.similarities.MatrixSimilarity(tfidf[categories_corpus])

In [22]:
td = tfidf[corpus[0]]
similar = index[td]

In [27]:
r = np.argsort(similar)[::-1]

In [28]:
r

array([11,  2,  7,  5, 18, 12, 17,  4,  1, 10,  6,  8,  3,  0, 15, 13,  9,
       16, 14], dtype=int64)

In [35]:
similar[7]

0.17693743